# n-Stacked gru

In [27]:
pip install --upgrade torch torchvision torchaudio

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import random
import os
import pandas as pd
import numpy as np
import datetime
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [25]:
# [Hyperparameter Setting]
CFG = {
    'TRAIN_WINDOW_SIZE': 90,  # 90일치로 학습
    'PREDICT_SIZE': 21,  # 21일치 예측
    'EPOCHS': 10,
    'LEARNING_RATE': 1e-4,
    'BATCH_SIZE': 1024,
    'SEED': 41,
    'HIDDEN_SIZE': 512,  # Hidden size of GRU model
    'STACKED_GRU_LAYERS': 3,  # Number of stacked GRU layers
}

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [5]:
train_data = pd.read_csv('data/train.csv').drop(columns=['ID', '제품'])

In [6]:
train_data

,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,1,3,2,0,0,2,0
2,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,B002-C001-0003,B002-C002-0008,B002-C003-0042,B002-03799,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15886,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,0,0,0,0,0,0,...,0,0,0,3,0,2,4,1,1,3
15887,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15888,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [7]:
# holiday effect df만들기

from datetime import datetime, timedelta

start_date = datetime(2022, 1, 1)
end_date = datetime(2023, 4, 4)

date_list = []
day_type_list = []

current_date = start_date
while current_date <= end_date:
    date_list.append(current_date.strftime('%Y-%m-%d'))
    day_type_list.append(1 if current_date.weekday() in [5, 6] else 0)
    current_date += timedelta(days=1)

# Create a DataFrame
date_df = pd.DataFrame({'Date': date_list, 'Day_Type': day_type_list})

print(date_df.tail())


           Date  Day_Type
454  2023-03-31         0
455  2023-04-01         1
456  2023-04-02         1
457  2023-04-03         0
458  2023-04-04         0


In [8]:
date_df

,Date,Day_Type
0,2022-01-01,1
1,2022-01-02,1
2,2022-01-03,0
3,2022-01-04,0
4,2022-01-05,0
...,...,...
454,2023-03-31,0
455,2023-04-01,1
456,2023-04-02,1
457,2023-04-03,0


In [9]:
holiday = train_data.copy()

In [10]:
holiday.drop(['브랜드', '대분류','중분류','소분류'], axis = 1, inplace = True)

In [11]:
i = 0
for date in holiday:
    holiday[date] = day_type_list[i]
    i +=1
    if i  > len(day_type_list):
        break
        

In [12]:
holiday

,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,1,1,0,0,0,0,0,1,1,0,...,1,0,0,0,0,0,1,1,0,0
1,1,1,0,0,0,0,0,1,1,0,...,1,0,0,0,0,0,1,1,0,0
2,1,1,0,0,0,0,0,1,1,0,...,1,0,0,0,0,0,1,1,0,0
3,1,1,0,0,0,0,0,1,1,0,...,1,0,0,0,0,0,1,1,0,0
4,1,1,0,0,0,0,0,1,1,0,...,1,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,1,1,0,0,0,0,0,1,1,0,...,1,0,0,0,0,0,1,1,0,0
15886,1,1,0,0,0,0,0,1,1,0,...,1,0,0,0,0,0,1,1,0,0
15887,1,1,0,0,0,0,0,1,1,0,...,1,0,0,0,0,0,1,1,0,0
15888,1,1,0,0,0,0,0,1,1,0,...,1,0,0,0,0,0,1,1,0,0


In [13]:
combined_df = pd.merge(train_data, holiday,left_index = True, right_index = True, how = 'left')
combined_df

,대분류,중분류,소분류,브랜드,2022-01-01_x,2022-01-02_x,2022-01-03_x,2022-01-04_x,2022-01-05_x,2022-01-06_x,...,2023-03-26_y,2023-03-27_y,2023-03-28_y,2023-03-29_y,2023-03-30_y,2023-03-31_y,2023-04-01_y,2023-04-02_y,2023-04-03_y,2023-04-04_y
0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
1,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
2,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
3,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
4,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,B002-C001-0003,B002-C002-0008,B002-C003-0042,B002-03799,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
15886,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
15887,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
15888,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0


In [14]:
train_data = combined_df.copy()

In [15]:
#categorical_columns = []
#for col in train_data:
#    if str(train_data[col].dtype) == 'object':
#        categorical_columns.append(col)

In [16]:
# Label Encoding
label_encoder = LabelEncoder()
categorical_columns = ['대분류', '중분류', '소분류', '브랜드']

for col in categorical_columns:
    label_encoder.fit(train_data[col])
    train_data[col] = label_encoder.transform(train_data[col])

In [17]:
train_data

,대분류,중분류,소분류,브랜드,2022-01-01_x,2022-01-02_x,2022-01-03_x,2022-01-04_x,2022-01-05_x,2022-01-06_x,...,2023-03-26_y,2023-03-27_y,2023-03-28_y,2023-03-29_y,2023-03-30_y,2023-03-31_y,2023-04-01_y,2023-04-02_y,2023-04-03_y,2023-04-04_y
0,1,6,37,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
1,2,7,43,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
2,2,7,43,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
3,2,7,43,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
4,0,0,2,2,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,2,7,41,3169,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
15886,2,7,43,3169,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
15887,2,7,43,3169,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
15888,2,7,43,3169,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0


In [18]:
def make_train_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
    data : 일별 판매량, 브랜드, 그리고 브랜드 키워드 카운트 정보가 있는 데이터 프레임
    train_size : 학습에 활용할 기간
    predict_size : 추론할 기간
    '''
    num_rows = len(data)
    holiday_start_index = data.columns.get_loc('2022-01-01_y')  # Find the index of the first '2022-01-01_y' column
    window_size = train_size + predict_size

    input_data = np.empty((num_rows * (459 - window_size + 1), train_size, 6)) # 6 features: 대분류, 중분류, 소분류, 브랜드, sales. holiday
    target_data = np.empty((num_rows * (459 - window_size + 1), predict_size))



    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])  # Extract encoding info ('대분류', '중분류', '소분류', '브랜드')
        sales_data = np.array(data.iloc[i, 4: holiday_start_index-1])
        holiday_data = np.array(data.iloc[i, holiday_start_index:])


        for j in range(len(sales_data) - window_size + 1):
            window = sales_data[j : j + window_size]
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size], holiday_data[j : j + train_size]))
            input_data[i * (459 - window_size + 1) + j] = temp_data
            target_data[i * (459 - window_size + 1) + j] = window[train_size:]


    return input_data, target_data

In [19]:
# 확인 완료

def make_predict_data(data, train_size=CFG['TRAIN_WINDOW_SIZE']):
    '''
    평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
    data : 일별 판매량, 브랜드, 그리고 holiday 정보가 있는 데이터 프레임
    train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간)
    '''
    num_rows = len(data)
    holiday_start_index = data.columns.get_loc('2022-01-01_y')  # Find the index of the first '2022-01-01_y' column
    
    input_data = np.empty((num_rows, train_size, 6))
    

    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])  # Extract encoding info ('대분류', '중분류', '소분류', '브랜드')
        sales_data = np.array(data.iloc[i, holiday_start_index-91:holiday_start_index-1])  # Extract daily sales data for the past 'train_size' days
        holiday_data = np.array(data.iloc[i, -train_size:])  # Extract holiday data for the past 'train_size' days

        window = sales_data[-train_size : ]
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size], holiday_data))
        #temp_data = np.column_stack((encode_info, sales_data, holiday_data))
        input_data[i] = temp_data
    
    return input_data


In [20]:
train_input, train_target = make_train_data(train_data)
test_input = make_predict_data(train_data)

  0%|          | 0/15890 [00:00<?, ?it/s]

  0%|          | 0/15890 [00:00<?, ?it/s]

In [21]:
# Train / Validation Split
data_len = len(train_input)
val_input = train_input[-int(data_len*0.2):]
val_target = train_target[-int(data_len*0.2):]
train_input = train_input[:-int(data_len*0.2)]
train_target = train_target[:-int(data_len*0.2)]

In [22]:
train_input.shape, train_target.shape, val_input.shape, val_target.shape, test_input.shape

((4436488, 90, 6),
 (4436488, 21),
 (1109122, 90, 6),
 (1109122, 21),
 (15890, 90, 6))

In [23]:
# [Custom Dataset]
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])

    def __len__(self):
        return len(self.X)

train_dataset = CustomDataset(train_input, train_target)
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_input, val_target)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [28]:


# [Stacked GRU Model]
class StackedGRUModel(nn.Module):
    def __init__(self, input_size=6, hidden_size=512, output_size=CFG['PREDICT_SIZE'], num_layers=CFG['STACKED_GRU_LAYERS']):
        super(StackedGRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden_size // 2, output_size)
        )
        self.actv = nn.ReLU()

    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size, x.device)

        # GRU layer
        gru_out, hidden = self.gru(x, hidden)

        # Only use the last output sequence
        last_output = gru_out[:, -1, :]

        # Fully connected layer
        output = self.actv(self.fc(last_output))

        return output.squeeze(1)

    def init_hidden(self, batch_size, device):
        # Initialize hidden state
        return torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device)

# [Model Training]
def train_stacked_gru(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = 9999999
    best_model = None

    for epoch in range(1, CFG['EPOCHS'] + 1):
        model.train()
        train_loss = []

        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)

            optimizer.zero_grad()

            output = model(X)
            loss = criterion(output, Y)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        val_loss = validation_stacked_gru(model, val_loader, criterion, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')

        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
            print('Model Saved')

    return best_model

def validation_stacked_gru(model, val_loader, criterion, device):
    model.eval()
    val_loss = []

    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.to(device)
            Y = Y.to(device)

            output = model(X)
            loss = criterion(output, Y)

            val_loss.append(loss.item())

    return np.mean(val_loss)

# [Run !!]
stacked_gru_model = StackedGRUModel()
optimizer = torch.optim.Adam(params=stacked_gru_model.parameters(), lr=CFG["LEARNING_RATE"])
trained_stacked_gru_model = train_stacked_gru(stacked_gru_model, optimizer, train_loader, val_loader, device)

# [Model Inference]
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

def inference(model, test_loader, device):
    predictions = []

    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.to(device)

            output = model(X)

            # Move model output to CPU and convert to numpy array
            output = output.cpu().numpy()

            predictions.extend(output)

    return np.array(predictions)

stacked_gru_pred = inference(trained_stacked_gru_model, test_loader, device)

# [Inverse Scaling and Post-processing]
for idx in range(len(stacked_gru_pred)):
    stacked_gru_pred[idx, :] = stacked_gru_pred[idx, :] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]

# Post-processing
stacked_gru_pred = np.round(stacked_gru_pred, 0).astype(int)

# [Submission]
submit = pd.read_csv('data/sample_submission.csv')
submit.iloc[:, 1:] = stacked_gru_pred
submit.to_csv('data/3-stacked_gru_weekend_submit.csv', index=False)


  0%|          | 0/4333 [00:00<?, ?it/s]

  0%|          | 0/1084 [00:00<?, ?it/s]

Epoch : [1] Train Loss : [81108.55767] Val Loss : [20467.11881]
Model Saved


  0%|          | 0/4333 [00:00<?, ?it/s]

KeyboardInterrupt: 